In [1]:
import os
import pandas as pd

# --- CẤU HÌNH ---
RAW_PATH = "../data"
OUTPUT_PATH = "../data"

MIN_USER_RATINGS = 5
MIN_MOVIE_RATINGS = 10
print("📂 Đang đọc dữ liệu CSV...")

ratings = pd.read_csv(os.path.join(RAW_PATH, "ratings.csv"))
movies  = pd.read_csv(os.path.join(RAW_PATH, "movies.csv"))
links   = pd.read_csv(os.path.join(RAW_PATH, "links.csv"))
print("🧹 Đang lọc dữ liệu...")

user_counts = ratings["userId"].value_counts()
movie_counts = ratings["movieId"].value_counts()
valid_users = user_counts[user_counts >= MIN_USER_RATINGS].index
valid_movies = movie_counts[movie_counts >= MIN_MOVIE_RATINGS].index

ratings_clean = ratings[
    ratings["userId"].isin(valid_users) &
    ratings["movieId"].isin(valid_movies)
][["userId", "movieId", "rating", "timestamp"]]
print(f"✅ Dữ liệu sạch còn lại: {len(ratings_clean)} dòng rating.")
print("🛠  Đang tạo file movieId–tmdbId...")
movie_tmdb = (
    ratings_clean[["movieId"]]
    .drop_duplicates()
    .merge(
        links[["movieId", "tmdbId"]],
        on="movieId",
        how="inner"
    )
    .dropna(subset=["tmdbId"])
    .sort_values("movieId")
)
output_path = os.path.join(OUTPUT_PATH, "movie_tmdb_filtered.csv")
movie_tmdb.to_csv(output_path, index=False)

print(f"✅ Đã lưu file tại: {output_path}")


📂 Đang đọc dữ liệu CSV...
🧹 Đang lọc dữ liệu...
✅ Dữ liệu sạch còn lại: 31842705 dòng rating.
🛠  Đang tạo file movieId–tmdbId...
✅ Đã lưu file tại: ../data\movie_tmdb_filtered.csv


import webview
import requests

API_KEY = "ff48b02cdcd1f6e40df93cb3ff292031"
TMDB_ID = 862

# Lấy trailer YouTube key từ TMDB
video_url = f"https://api.themoviedb.org/3/movie/{TMDB_ID}/videos"
params = {"api_key": API_KEY}
res = requests.get(video_url, params=params)
data = res.json()
print(data)
youtube_key = None
for v in data["results"]:
    if v["site"] == "YouTube" and v["type"] == "Trailer":
        youtube_key = v["key"]
        break

if not youtube_key:
    print("❌ Không tìm thấy trailer")
    exit()



In [2]:
import requests

API_KEY = "ff48b02cdcd1f6e40df93cb3ff292031"

url = "https://api.themoviedb.org/3/movie/862"
params = {
    "api_key": API_KEY,
    "language": "vi-VN"
}

r = requests.get(url, params=params)

print("Status:", r.status_code)
print(r.json())


Status: 200
{'adult': False, 'backdrop_path': '/3Rfvhy1Nl6sSGJwyjb0QiZzZYlB.jpg', 'belongs_to_collection': {'id': 10194, 'name': 'Câu Chuyện Đồ Chơi', 'poster_path': '/rki5qLuwb0xnnE9seehxO9TlLhW.jpg', 'backdrop_path': '/hApclyB9NEZEQujAVajzi5iWE4a.jpg'}, 'budget': 30000000, 'genres': [{'id': 10751, 'name': 'Phim Gia Đình'}, {'id': 35, 'name': 'Phim Hài'}, {'id': 16, 'name': 'Phim Hoạt Hình'}, {'id': 12, 'name': 'Phim Phiêu Lưu'}], 'homepage': '', 'id': 862, 'imdb_id': 'tt0114709', 'origin_country': ['US'], 'original_language': 'en', 'original_title': 'Toy Story', 'overview': 'Thế giới đồ chơi của cậu bé Andy bị xáo trộn khi có sự xuất hiện của cảnh sát vũ trụ Buzz Lightyear. Cảm thấy mình bị “thất sủng”, anh chàng cao bồi gỗ Woody xô xát với Buzz và cả hai bị rơi vào một cuộc phiêu lưu đầy mạo hiểm. Câu chuyện nói về thế giới đồ chơi của cậu bé...', 'popularity': 22.2449, 'poster_path': '/2k5424igOcC1rjJoME8WNREEOgy.jpg', 'production_companies': [{'id': 3, 'logo_path': '/1TjvGVDMYsj6J

In [3]:
noPoster = []
noBackdrop = []
noLogo = []
noInfo = []

In [4]:
import csv
import time
import requests

API_KEY = "ff48b02cdcd1f6e40df93cb3ff292031"

INPUT_CSV = "../data/movie_tmdb.csv"
OUTPUT_CSV = "../data/movie_info.csv"

BASE_URL = "https://api.themoviedb.org/3/movie/{}"

# --- GHI HEADER ---
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8-sig") as f_out:
    writer = csv.writer(f_out)
    writer.writerow([
        "movieId",
        "tmdbId",
        "title",
        "original_title",
        "release_date",
        "vote_average",
        "vote_count",
        "genres",
        "overview"
    ])

# --- DUYỆT FILE INPUT ---
with open(INPUT_CSV, "r", encoding="utf-8") as f_in:
    reader = csv.DictReader(f_in)

    for i, row in enumerate(reader, start=1):
        movie_id = row["movieId"]

        # tmdbId có thể là 862.0 → ép về int
        tmdb_id = row["tmdbId"]
        if not tmdb_id or tmdb_id == "":
            continue

        tmdb_id = int(float(tmdb_id))

        url = BASE_URL.format(tmdb_id)
        params = {
            "api_key": API_KEY,
            "language": "vi-VN"
        }

        try:
            r = requests.get(url, params=params, timeout=10)

            if r.status_code != 200:
                print(f"[WARN] movieId={movie_id}, tmdbId={tmdb_id}, status={r.status_code}")
                continue

            data = r.json()

            genres = ", ".join([g["name"] for g in data.get("genres", [])])

            row_out = [
                movie_id,
                tmdb_id,
                data.get("title"),
                data.get("original_title"),
                data.get("release_date"),
                data.get("vote_average"),
                data.get("vote_count"),
                genres,
                data.get("overview")
            ]

            with open(OUTPUT_CSV, "a", newline="", encoding="utf-8-sig") as f_out:
                writer = csv.writer(f_out)
                writer.writerow(row_out)

            print(f"[OK] {i}: movieId={movie_id}, tmdbId={tmdb_id}")

            # tránh bị rate-limit
            time.sleep(0.25)

        except Exception as e:
            print(f"[ERROR] movieId={movie_id}, tmdbId={tmdb_id}: {e}")


FileNotFoundError: [Errno 2] No such file or directory: '../data/movie_tmdb.csv'

In [ ]:
import csv
import os
import time
import json
import requests

API_KEY = "ff48b02cdcd1f6e40df93cb3ff292031"
BASE_URL = "https://api.themoviedb.org/3"
POSTER_BASE = "https://image.tmdb.org/t/p/w500"
BACKDROP_BASE = "https://image.tmdb.org/t/p/w1280"
LOGO_BASE = "https://image.tmdb.org/t/p/original"

HEADERS = {
    "User-Agent": "Mozilla/5.0"
}

POSTER_DIR = "../data/image/poster"
BACKDROP_DIR = "../data/image/backdrop"
LOGO_DIR = "../data/image/logo"

os.makedirs(POSTER_DIR, exist_ok=True)
os.makedirs(BACKDROP_DIR, exist_ok=True)
os.makedirs(LOGO_DIR, exist_ok=True)



def download_image(path, save_path, base_url, retries=3):
    if not path:
        return False

    url = base_url + path

    for _ in range(retries):
        try:
            r = requests.get(
                url,
                headers=HEADERS,
                stream=True,
                timeout=20
            )

            if r.status_code == 200:
                with open(save_path, "wb") as f:
                    for chunk in r.iter_content(1024):
                        f.write(chunk)
                return True

        except requests.exceptions.RequestException:
            time.sleep(2)

    return False



with open("../data/movie_tmdb.csv", newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)

    for row in reader:
        movie_id = row["movieId"]
        tmdb_id = row["tmdbId"]

        # ===== BỎ QUA PHIM CŨ =====
        if int(movie_id) < 6011:
            continue

        print(f"🎬 movieId={movie_id}")

        # ===== Movie info =====
        movie_url = f"{BASE_URL}/movie/{tmdb_id}"
        r = requests.get(
            movie_url,
            params={"api_key": API_KEY, "language": "vi-VN"},
            headers=HEADERS,
            timeout=15
        )

        if r.status_code != 200:
            print("Không lấy được movie info")
            noInfo.append(movie_id)
            continue

        data = r.json()

        # ===== Poster =====
        if data.get("poster_path"):
            ok = download_image(
                data["poster_path"],
                os.path.join(POSTER_DIR, f"{movie_id}.jpg"),
                POSTER_BASE
            )

            if not ok:
                noPoster.append(movie_id)
        else:
            noPoster.append(movie_id)

        # ===== Backdrop =====
        if data.get("backdrop_path"):
            ok = download_image(
                data["backdrop_path"],
                os.path.join(BACKDROP_DIR, f"{movie_id}.jpg"),
                BACKDROP_BASE
            )

            if not ok:
                noBackdrop.append(movie_id)
        else:
            noBackdrop.append(movie_id)

        # ===== Logo =====
        images_url = f"{BASE_URL}/movie/{tmdb_id}/images"
        r = requests.get(
            images_url,
            params={"api_key": API_KEY},
            headers=HEADERS,
            timeout=15
        )

        if r.status_code == 200:
            logos = r.json().get("logos", [])
            if logos:
                logo = next(
                    (l for l in logos if l.get("iso_639_1") == "en"),
                    logos[0]
                )
                ok = download_image(
                    logo["file_path"],
                    os.path.join(LOGO_DIR, f"{movie_id}.png"),
                    LOGO_BASE
                )
                if not ok:
                    noLogo.append(movie_id)
            else:
                noLogo.append(movie_id)
        else:
            noLogo.append(movie_id)

        time.sleep(0.5)


# =====================
# GHI KẾT QUẢ RA FILE
# =====================
with open("missing_images.json", "w", encoding="utf-8") as f:
    json.dump(
        {
            "noPoster": noPoster,
            "noBackdrop": noBackdrop,
            "noLogo": noLogo
        },
        f,
        ensure_ascii=False,
        indent=2
    )
print("No poster:", len(noPoster))
print("No backdrop:", len(noBackdrop))
print("No logo:", len(noLogo))
